---
layout: post

title: 백준 문제 중 3190번 with python, 구현, 뱀

category: Algorithms
---


### 백준 문제 중 3190번
<a>https://www.acmicpc.net/problem/3190</a>

### 문제

Dummy' 라는 도스게임이 있다. 이 게임에는 뱀이 나와서 기어다니는데, 사과를 먹으면 뱀 길이가 늘어난다. 뱀이 이리저리 기어다니다가 벽 또는 자기자신의 몸과 부딪히면 게임이 끝난다.

게임은 NxN 정사각 보드위에서 진행되고, 몇몇 칸에는 사과가 놓여져 있다. 보드의 상하좌우 끝에 벽이 있다. 게임이 시작할때 뱀은 맨위 맨좌측에 위치하고 뱀의 길이는 1 이다. 뱀은 처음에 오른쪽을 향한다.

뱀은 매 초마다 이동을 하는데 다음과 같은 규칙을 따른다.

+ 먼저 뱀은 몸길이를 늘려 머리를 다음칸에 위치시킨다.
만약 이동한 칸에 사과가 있다면, 그 칸에 있던 사과가 없어지고 꼬리는 움직이지 않는다.
+ 만약 이동한 칸에 사과가 없다면, 몸길이를 줄여서 꼬리가 위치한 칸을 비워준다. 즉, 몸길이는 변하지 않는다.
+ 사과의 위치와 뱀의 이동경로가 주어질 때 이 게임이 몇 초에 끝나는지 계산하라.

### 입력

첫째 줄에 보드의 크기 N이 주어진다. (2 ≤ N ≤ 100) 다음 줄에 사과의 개수 K가 주어진다. (0 ≤ K ≤ 100)

다음 K개의 줄에는 사과의 위치가 주어지는데, 첫 번째 정수는 행, 두 번째 정수는 열 위치를 의미한다. 사과의 위치는 모두 다르며, 맨 위 맨 좌측 (1행 1열) 에는 사과가 없다.

다음 줄에는 뱀의 방향 변환 횟수 L 이 주어진다. (1 ≤ L ≤ 100)

다음 L개의 줄에는 뱀의 방향 변환 정보가 주어지는데,  정수 X와 문자 C로 이루어져 있으며. 게임 시작 시간으로부터 X초가 끝난 뒤에 왼쪽(C가 'L') 또는 오른쪽(C가 'D')로 90도 방향을 회전시킨다는 뜻이다. X는 10,000 이하의 양의 정수이며, 방향 전환 정보는 X가 증가하는 순으로 주어진다.

### 출력

첫째 줄에 게임이 몇 초에 끝나는지 출력한다.

---

처음 문제를 보고 풀때 재귀적으로 dfs를 구현했더니 Recursion error가 발생했다. 문제에 제시된 제한은 없지만 이상하게 오류가 발생했고 재귀구조를 반복구조로 개선했더니 정답처리가 되었다.

까다로웠던 부분이 두 가지정도 있었다.   
첫번째는 방향이 바뀌는것을 처리하는것이다. 이전까지 접해봤던 문제들과 다르게 주어지는 방향은 L과 D로 두개지만 현재 진행하던 방향을 기준으로 다음 방향이 달라진다.  
두번째는 게임이 종료되는 정확한 시점이 헷갈렸는데 벽직전까지는 진행이 가능하고 벽 직전 부분에서 벽 방향으로 진행한 시간까지 시간를 센 후 종료해야한다.

코드의 흐름은 다음과 같다.  

1. 입력을 처리하고 맵 전체를 0으로 초기화 한뒤 사과가 있는 부분은 1을 대입한다. 단 주어지는 입력 때문에(1,1 시작) 맵의 크기를 n+1 x n+1로 한다.
2. 시간에 따라 달라질 진행방향을 defaultdict에 저장한다.  
3. 진행이 가능한 방향 오른쪽, 아래, 왼쪽, 위 를 각각(0, 1), (1, 0), (0, -1),(-1, 0)의 튜플형태를 키로하고 'L', 'D', 'S' 를 값으로 갖는 딕셔너리 dir_cases를 선언한다. L은 왼쪽 D는 오른쪽 S를 이전과 같은방향을 뜻한다.  
 + 예를 들어 dir_cases[(0,1)]['L'] 은 현재 진행하던 방향이 동쪽이고 다음 진행할 방향은 동쪽 진행을 기준으로 왼쪽인 (-1, 0)을 나타낸다
4.[1, 1]을 초기값으로 갖는 큐 snake안에 다음으로 탐색을 진행할[nr, nc]이 있는지와 범위를 벗어나 벽에 부딪히는지 살피면서 dfs를 진행한다.
5. 만약 board[nr][nc]가 0이라면 사과가 없으므로 snake.popleft()를 해서 꼬리를 지운다. 1이라면 사과가를 먹으므로 꼬리칸 제거를 건너뛰고 board[nr][nc]에 0을 대입해 사과를 없애준다.
6. 게임 종료조건이 된다면 time + 1을 출력한 뒤 GAME_START를 False로 바꿔준다.

In [25]:
from collections import deque, defaultdict

 # 입력처리 및 맵 초기화
n = int(input())
k = int(input())
apples = []
for _ in range(k):
    apples.append(list(map(int, input().split())))

board = [[0 for _ in range(0, n+1)] for _ in range(0, n+1)]
for i,j in apples: board[i][j] = 1

l = int(input())
c = []
for _ in range(l):
    a, b = list(input().split())
    c.append([int(a), b.upper()])

 # 진행 방향의 변화를 저장할 딕셔너리
changes = defaultdict(int)
for i,d in c:
    changes[i] = d


 # 현재 진행 방향에 따라 다음 진행 방향을 참조할 딕셔너리 선언
dir_cases = {
(0, 1): {'S': (0, 1), 'L':(-1, 0), 'D':(1, 0)},
(1, 0): {'S': (1, 0), 'L':(0, 1), 'D':(0, -1)},
(0, -1): {'S': (0, -1), 'L':(1, 0), 'D':(-1, 0)},
(-1, 0): {'S': (-1, 0), 'L':(0, -1), 'D':(0, 1)},
}



def dfs():
  
     # 게임 시작전 초기화
    time = 0
    GAME_START = True
    snake = deque()
    snake.append([1, 1])
    r, c = nr, nc =1, 1
    moving_dir = nd = (0, 1)

     # 게임이 진행되는 동안 반복
    while GAME_START:
        print(snake)

         # 다음 진행 방향을 결정하는 처리
        moving_dir = nd
        next_dir = changes[time] if changes[time] else 'S'
        nd = dir_cases[moving_dir][next_dir]
        nr, nc = nr + nd[0], nc + nd[1]
       
         # 벽과 만난다면 게임 종료 
        if not (1 <= nr <= n and 1 <= nc <= n):
            print(time+1)
            GAME_START = False
            return
   
         # 뱀 자기 자신과 만나다면 종료    
        if [nr, nc] in snake:
            print(time+1)
            GAME_START = False
            return
        else:
             # 머리를 진행하고 사과가 없다면 꼬리삭제
            snake.append([nr, nc])
            if board[nr][nc] == 0:
                snake.popleft()   
            else:
                board[nr][nc] = 0
        time += 1

dfs()

10
5
1 5
1 3
1 2
1 6
1 7
4
8 D
10 D
11 D
13 L
deque([[1, 1]])
deque([[1, 1], [1, 2]])
deque([[1, 1], [1, 2], [1, 3]])
deque([[1, 2], [1, 3], [1, 4]])
deque([[1, 2], [1, 3], [1, 4], [1, 5]])
deque([[1, 2], [1, 3], [1, 4], [1, 5], [1, 6]])
deque([[1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7]])
deque([[1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [1, 8]])
deque([[1, 4], [1, 5], [1, 6], [1, 7], [1, 8], [1, 9]])
deque([[1, 5], [1, 6], [1, 7], [1, 8], [1, 9], [2, 9]])
deque([[1, 6], [1, 7], [1, 8], [1, 9], [2, 9], [3, 9]])
deque([[1, 7], [1, 8], [1, 9], [2, 9], [3, 9], [3, 8]])
deque([[1, 8], [1, 9], [2, 9], [3, 9], [3, 8], [2, 8]])
13
